In [1]:
import numpy as np
import os
from scipy.misc import imread
import cPickle as pkl
import scipy.signal
import numpy.fft as fft
import sys
import optparse
from PIL import Image
import re
import itertools
from scipy import ndimage

import time
import datetime

import pandas as pd

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure
import csv

import pymworks 
import pandas as pd
import operator
import codecs

import scipy.io
import copy# Abstract struct class       
class Struct:
    def __init__ (self, *argv, **argd):
        if len(argd):
            # Update by dictionary
            self.__dict__.update (argd)
        else:
            # Update by position
            attrs = filter (lambda x: x[0:2] != "__", dir(self))
            for n in range(len(argv)):
                setattr(self, attrs[n], argv[n])


class cycstruct(Struct):
    times = []
    idxs = 0
    vals = 0
    ordernum = 0
    triggers = 0


In [2]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from parse_mw_events import get_bar_events

In [4]:
prepend = '/' #'/Users/julianarhee'
#source_dir = os.path.join(prepend,'nas/volume1/2photon/RESDATA/20170724_CE051/retinotopy1') #options.source_dir #'/nas/volume1/2photon/RESDATA/TEFO/20160118_AG33/fov1_gratings1'
#source_dir = os.path.join(prepend,'nas/volume1/2photon/RESDATA/test_triggers/timer') #options.source_dir #'/nas/volume1/2photon/RESDATA/TEFO/20160118_AG33/fov1_gratings1'
source_dir = os.path.join(prepend,'nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3') #options.source_dir #'/nas/volume1/2photon/RESDATA/TEFO/20160118_AG33/fov1_gratings1'

stimtype = 'bar' #options.stimtype #'grating'
mask = False #options.mask # False
long_trials = False #options.long_trials #True
no_ard = False #options.no_ard
frame_rate = 44.63 #85.14

# Look in child dir (of source_dir) to find mw_data paths:
mw_files = os.listdir(os.path.join(source_dir, 'mw_data'))
mw_files = [m for m in mw_files if m.endswith('.mwk')]

In [5]:
mwfile = mw_files[0]

fn_base = mwfile[:-4]
mw_data_dir = os.path.join(source_dir, 'mw_data')
mw_fn = fn_base+'.mwk'
dfn = os.path.join(mw_data_dir, mw_fn)
dfns = [dfn]

print "MW file: ", dfns

MW file:  ['/nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3/mw_data/20170811_CE052_run3.mwk']


In [6]:
if stimtype=='bar':
    pixelevents, stimevents, trigger_times, session_info = get_bar_events(dfns[0])
else:
    pixelevents, stimevents, trialevents, trigger_times, session_info = get_stimulus_events(dfns, stimtype=stimtype)


****************************************************************
Parsing file
/nas/volume1/2photon/RESDATA/20170811_CE052/retinotopy3/mw_data/20170811_CE052_run3.mwk... 
Found 1 start events in session.
Bounds:  [[457767409969, 458175153294]]
bound ID: 0 407.743325 sec
****************************************************************
................................................................
SECTION 0
................................................................
Found > 1 name for frame-trigger:
Choose:  ['frame_trigger_timeout', 'frame_trigger_on', 'trigger_received', 'trigger_active', 'frame_trigger']
Hint: RSVP could be FrameTrigger, otherwise frame_trigger.
Type var name to use: frame_trigger
first on event:  Event[code=39, name=frame_trigger, time=457780845705, value=0]
first off event:  Event[code=39, name=frame_trigger, time=457855277621, value=1]
Duration of first run: 74.4319 sec.
[Event[code=39, name=frame_trigger, time=457767410045, value=1], Event[code=39, name=fram

In [7]:
print len(trigger_times)
print frame_rate

4
44.63


In [8]:
print session_info
n_expected_triggers = (1./session_info['target_freq'])*session_info['ncycles']*frame_rate*len(trigger_times)
print "Expected {0:.2f} frame triggers.".format(n_expected_triggers)

{'ncycles': 20, 'barwidth': 2.5, 'target_freq': 0.27}
Expected 13223.70 frame triggers.


In [9]:
no_ard = False

In [11]:
#if no_ard is False:
# --------------------------------------------------------------------
# ARDUINO codes:
# --------------------------------------------------------------------

ard_fn = fn_base+'.txt'
ard_data_dir = os.path.join(source_dir, 'ard_data')
ard_dfn = os.path.join(ard_data_dir, ard_fn)

f = codecs.open(ard_dfn, 'r')
data = f.read()
bad_end = False
if not data[-1]=='_':
   bad_end = True
packets = data.split('__')
nsplits = packets[0].count('*')
tmp_ard_evs = []
for pidx,packet in enumerate(packets):
    #print pidx
    if pidx==0:
        packet = packet[1:]
    elif pidx==len(packets)-1:
        if bad_end is True:
          continue
        else:
          packet = packet[:-1]
    try:
        if nsplits == 2:
            pin = packet.split('*', 2)[0]
            tstring = packet.split('*', 2)[2]
        else:
            pin = packet.split('*', 1)[0]
            tstring = packet.split('*', 1)[1]
        try:
            pin_id = int(pin) #get_int(pin) #pin #int(pin)
            #pin_id = sum(ord(c) << (i * 8) for i, c in enumerate(pin[::-1]))
            tstamp = int(tstring) #sum(ord(c) << (i * 8) for i, c in enumerate(tstring[::-1]))
            tmp_ard_evs.append([pin_id, tstamp])
        except NameError as e:
            print pidx
            print pin_id
            print tstring

    except IndexError as e:
        print pidx
        print packet
print packets[0:5]
print nsplits

['_0*0*59564076', '0*0*59565052', '0*0*59566096', '0*0*59567108', '0*0*59568152']
2


In [12]:
print len(packets)

310923


In [16]:
# Get lists of arduino times and codes:
ard_times = np.array([i[1] for i in tmp_ard_evs])
ard_codes = np.array([i[0] for i in tmp_ard_evs])
print tmp_ard_evs[0:5]

[[0, 59564076], [0, 59565052], [0, 59566096], [0, 59567108], [0, 59568152]]


In [17]:
# Check for arduino roll-overs:
tdiffs = np.array(np.diff(ard_times))
rollover_idxs = [t for t in tdiffs if t<0] #np.where(tdiffs<0)
if any(rollover_idxs):
    print "Found rollover points!  Fix this..."

print rollover_idxs

[]


In [18]:
# PLOT intervals:
tstamp_intervals = np.array(np.diff(ard_times))
print "N intervals:", len(tstamp_intervals)
print "min:", min(tstamp_intervals)
print "max:", max(tstamp_intervals)
print len([i for i,v in enumerate(tstamp_intervals) if v>15000])
p1 = figure(title="Intervals between frame-on triggers (us)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

#medintervals = np.array([i for i in intervals if i<=10000000])

hist, edges = np.histogram(tstamp_intervals, density=True, bins=5000)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

N intervals: 310921
min: 976
max: 1368
0


In [19]:
offs = [i for i,v in enumerate(tmp_ard_evs) if v[0]==0]
print len(offs)
off_times = [tmp_ard_evs[e][1] for e in offs]
off_intervals = np.diff(off_times)[0:100]
print min(off_intervals)
print max(off_intervals)

103036
976
1196


In [20]:
#tmp_onset_idxs = np.where(np.diff([e[0] for e in ard_evs]) > 0)[0]   # where trigger val changes from 0-->1
#tmp_triggers = [e[0] for e in ard_evs]
on_val = 1
off_val = 0
tmp_onset_idxs = np.where(np.diff([e[0] for e in tmp_ard_evs]) > off_val)[0]  # onsets are where trigger val changes from 0-->1
first_onset_idx = tmp_onset_idxs[0]+1                               # Get first SI frame-trigger event
ard_evs = tmp_ard_evs[first_onset_idx:]                                  # Get rid of extra junk at start before DI trigger from SI
start_idx = 0 #copy.copy(first_onset_idx)
print ard_evs[start_idx]

frame_trigger_evs = []
frame_idxs = []
while start_idx < len(ard_evs)-1:
    found_new_frames = False
    curr_chunk = ard_evs[start_idx:start_idx+20]
    try:
        next_offset_ev = next(off for off in curr_chunk if off[0]==off_val)
        next_onset_ev = next(on for on in curr_chunk if on[0]==on_val and on[1]>next_offset_ev[1])
        found_off_idx = [i[1] for i in ard_evs].index(next_offset_ev[1])
        last_found_idx = [i[1] for i in ard_evs].index(next_onset_ev[1])
        frame_trigger_evs.append([ard_evs[start_idx], next_offset_ev])
        frame_idxs.append((start_idx, found_off_idx))
        start_idx = last_found_idx
        found_new_frame=True
    except StopIteration:
        print "No more triggers."
        if found_new_frame is True:  # No offset found
            print "last start idx: ", start_idx
        break
            

[1, 69919972]
No more triggers.
last start idx:  24713


In [21]:
print "current ON ev: ", ard_evs[start_idx] 
print "next offset ev: ", next_offset_ev
print "next onset ev: ", next_onset_ev
print frame_trigger_evs[-1]
if ard_evs[start_idx][1] > frame_trigger_evs[-1][1][1]:
    print "Appending last on-off pair."
    frame_trigger_evs.append([ard_evs[start_idx], next_offset_ev])


current ON ev:  [1, 100151156]
next offset ev:  [0, 100149880]
next onset ev:  [1, 100151156]
[[1, 100139680], [0, 100149880]]
Appending last on-off pair.


In [22]:
all_frame_trigger_evs = [val for sublist in frame_trigger_evs for val in sublist]
print "N fr trigger events: ", len(frame_trigger_evs)
print "N total trigger events: ", len(all_frame_trigger_evs)
print frame_trigger_evs[0:5]
5873*2

N fr trigger events:  2317
N total trigger events:  4634
[[[1, 69919972], [0, 69930680]], [[1, 69931872], [0, 69942580]], [[1, 69943772], [0, 69954480]], [[1, 69955672], [0, 69966380]], [[1, 69967572], [0, 69988992]]]


11746

In [23]:
n_expected_triggers = (1./session_info['target_freq'])*session_info['ncycles']*frame_rate*len(trigger_times)
print "Frame rate: ", frame_rate
print "N on-off pairs found:", len(frame_trigger_evs)
print "Expected {0:.2f} frame triggers. Missing {missed} triggers.".format(n_expected_triggers, missed=round(n_expected_triggers)-len(frame_trigger_evs)*2.)


Frame rate:  44.63
N on-off pairs found: 2317
Expected 13223.70 frame triggers. Missing 8590.0 triggers.


In [24]:
frame_trigger_evs[0][0]

[1, 69919972]

In [25]:
# PLOT time between each frame:
frame_intervals = np.diff([fr[0][1] for fr in frame_trigger_evs])
print "N intervals:", len(frame_intervals)
print "min:", min(frame_intervals)
print "max:", max(frame_intervals)
frame_high = (1/frame_rate)*1E6
print "frame dur: ", framedur

wiggle_room = 200 # time in us that we expect variation
misses = [i for i,v in enumerate(frame_intervals) if v>frame_high+200]
print "Skipped off triggers on %i frames." % len(misses)
p1 = figure(title="Intervals between frame-on triggers (us)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

hist, edges = np.histogram(frame_intervals, density=True, bins=5000)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

N intervals: 2316
min: 10704
max: 24312
frame dur: 

NameError: name 'framedur' is not defined

In [334]:
print "misses:", misses[0:10]
print frame_trigger_evs[misses[0]]
missed_idx = misses[0]

# Find the corresponding frame-start index from the full list of arduiono-frame-events.
# Since an OFF trigger was missed, there will be many more "frame HIGH" events between a given
# missed-trigger index and it's corresponding next-found OFF index.
ard_idx_start = [ard[1] for ard in ard_evs].index(frame_trigger_evs[missed_idx][0][1])
curr_ard_chunk = ard_evs[ard_idx_start:ard_idx_start+40]
print curr_ard_chunk

misses: [30, 36, 42, 48, 54, 60, 107, 113, 119, 125]
[[1, 13006636], [0, 13028056]]
[[1, 13006636], [1, 13007828], [1, 13009016], [1, 13010208], [1, 13011396], [1, 13012588], [1, 13013776], [1, 13014968], [1, 13016156], [1, 13017348], [1, 13018536], [1, 13019728], [1, 13020916], [1, 13022108], [1, 13023296], [1, 13024488], [1, 13025676], [1, 13026868], [0, 13028056], [1, 13029248], [1, 13030436], [1, 13031628], [1, 13032816], [1, 13034008], [1, 13035196], [1, 13036388], [1, 13037584], [1, 13038768], [0, 13039956], [1, 13041148], [1, 13042336], [1, 13043528], [1, 13044716], [1, 13045908], [1, 13047096], [1, 13048288], [1, 13049476], [1, 13050668], [0, 13051856], [1, 13053048]]


In [346]:
13041148 - 13029248
13053048-13041148

11900

In [347]:
np.diff([i[1] for i in curr_ard_chunk])

array([1192, 1188, 1192, 1188, 1192, 1188, 1192, 1188, 1192, 1188, 1192,
       1188, 1192, 1188, 1192, 1188, 1192, 1188, 1192, 1188, 1192, 1188,
       1192, 1188, 1192, 1196, 1184, 1188, 1192, 1188, 1192, 1188, 1192,
       1188, 1192, 1188, 1192, 1188, 1192])

In [300]:
idx_in_ardevs = [fr[1] for fr in ard_evs].index(frame_on_evs[30][1])
idx_in_frevs = [fr[1] for fr in frame_trigger_evs].index(frame_on_evs[30][1])
print ard_evs[idx_in_ardevs]
print frame_trigger_evs[idx_in_frevs]
print ard_evs[idx_in_ardevs-1:idx_in_ardevs+40]
# print frame_trigger_evs[idx_in_frevs-1:idx_in_frevs+10]

[1, 13006636]
[1, 13006636]
[[0, 13005448], [1, 13006636], [1, 13007828], [1, 13009016], [1, 13010208], [1, 13011396], [1, 13012588], [1, 13013776], [1, 13014968], [1, 13016156], [1, 13017348], [1, 13018536], [1, 13019728], [1, 13020916], [1, 13022108], [1, 13023296], [1, 13024488], [1, 13025676], [1, 13026868], [0, 13028056], [1, 13029248], [1, 13030436], [1, 13031628], [1, 13032816], [1, 13034008], [1, 13035196], [1, 13036388], [1, 13037584], [1, 13038768], [0, 13039956], [1, 13041148], [1, 13042336], [1, 13043528], [1, 13044716], [1, 13045908], [1, 13047096], [1, 13048288], [1, 13049476], [1, 13050668], [0, 13051856], [1, 13053048]]


In [305]:
frame_high

11745.360582569885

In [312]:
print frame_on_evs[30]
print frame_on_evs[30][1] + np.floor(frame_high)
ard_evs[idx_in_ardevs:idx_in_ardevs+20]

[1, 13006636]
13018381.0
155.0
-1033.0
10708


In [254]:
on_tstamps = [t[1] for i,t in enumerate(tmp_ard_evs) if t[0]==1 and tmp_ard_evs[i-1][0]==0]
off_tstamps = [t[1] for i,t in enumerate(tmp_ard_evs) if t[0]==0 and tmp_ard_evs[i-1][0]==1]

on_idxs = [i+1 for i,val in enumerate(np.diff([i[0] for i in tmp_ard_evs])) if val==1]
off_idxs = [i+1 for i,val in enumerate(np.diff([i[0] for i in tmp_ard_evs])) if val==-1]
on_tstamps = [tmp_ard_evs[idx][1] for idx in on_idxs]
off_tstamps = [tmp_ard_evs[idx][1] for idx in off_idxs]

print "n on events: ", len(on_tstamps)
print "n off events:", len(off_tstamps)
for idx,(on,off) in enumerate(zip(on_tstamps, off_tstamps)):
    if on > off:
        print idx
        
#frame_durs = [off-on for on,off in zip(on_tstamps, off_tstamps)]
frame_intervals = np.diff(on_tstamps)
missing_between = [i for i in frame_intervals if i>15000]
print "frame high: ", frame_high

print len(missing_between)
print frame_intervals[0:100]

n on events:  11376
n off events: 11376
frame high:  11745.3605826


In [248]:
p1 = figure(title="Intervals between frame-on triggers (us)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

hist, edges = np.histogram(np.array(np.diff(on_tstamps)), density=True, bins=5000)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [214]:
onset_idxs = [i+1 for i in tmp_onset_idxs]
tmp_offset_idxs = np.where(np.diff([e[0] for e in ard_evs]) < 0)[0]  # where trigger val changes from 1-->0
offset_idxs = [t+1 for t in tmp_offset_idxs]

if len(onset_idxs) != len(offset_idxs):
    print "N offsets %i does not match N onsets %i." % (len(offset_idxs), len(onset_idxs))
    #offsets.append(len(ard_evs)-1)
else:
    print "N offsets and onsets match! Found %i frame events." % len(onsets)

print "Expected {0:.2f} frame triggers. Missing {missed} triggers.".format(n_expected_triggers, missed=round(n_expected_triggers)-len(onset_idxs))


NameError: name 'onsets' is not defined

In [48]:
no_off_trigger = False
if ard_evs[-1][0]==1:
    no_off_trigger = True

In [50]:
offset_idxs = [a[1] for a in ard_evs if a[0]==0]
misses = [i for i,v in enumerate(np.diff(offset_idxs)[0:100]/1E3) if v>12]
print len(misses)

42


In [51]:
# intervals = np.diff([i[1] for i in ard_evs])
intervals = np.diff([i[1] for i in ard_evs])
print len(intervals)

132814


In [52]:
# PLOT intervals:
p1 = figure(title="Intervals between frame-on triggers (us)",tools="save, zoom_in, zoom_out, pan",
            background_fill_color="white")

#medintervals = np.array([i for i in intervals if i<=10000000])

hist, edges = np.histogram(intervals, density=True, bins=1000)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")
show(p1)

In [27]:
min(intervals)

1172

In [ ]:
tr_start_idx = onsets[0]
too_short = 500 #500 # if diff b/w offsets is < 100, interval is only 100*1500us = 150ms... 
on_intervals = np.where(np.diff(onsets) > too_short)[0] #[t for t in np.diff(onsets) if t>too_short]
#off_intervals = np.where(np.diff(offsets) > too_short)[0]

    new_file_idxs = [onsets[i+1] for i in on_intervals]
    new_file_idxs.append(tr_start_idx)
    new_file_idxs = sorted(new_file_idxs)
    new_file_markers = [ard_evs[i] for i in sorted(new_file_idxs)]

    end_file_idxs = []
    for i,nidx in enumerate(new_file_idxs[0:-1]):
        sublist = ard_evs[nidx:new_file_idxs[i+1]]
        last_off = np.where(np.diff([e[0] for e in sublist]) < 0)[0][-1]
        off_idx = nidx + last_off + 1
        end_file_idxs.append(off_idx)
    if no_off_trigger:
        end_file_idxs.append(len(ard_evs)-1) # just add the last trigger event for end-of-file
    else:
        sublist = ard_evs[onsets[-1]:] # Find all evs from last onset onward.
        final_trigger = np.where(np.diff([e[0] for e in sublist]) < 0)[0][0] # Find first off-trigger
        final_trigger_idx = onsets[-1] + final_trigger + 1 # add 2 to get actual off-trigger event after the 1st <0 difference found
        end_file_idxs.append(final_trigger_idx)

    end_file_markers = [ard_evs[i] for i in sorted(end_file_idxs)]
    print "Found %i TIF file chunks." % len(new_file_idxs)
    if len(new_file_markers)==1:
        print "DUR of session is: %f sec." % ((end_file_markers[0][1] - new_file_markers[0][1])/1E6)
    else:
        for midx in range(len(new_file_markers)):
            print "DUR of chunk is: %0.2f sec." % ((end_file_markers[midx][1] - new_file_markers[midx][1])/1E6)



    # # Plot to check file chunks
    # plt.plot([e[0] for e in evs], 'k*')
    # plt.plot(new_file_idxs, np.ones((1,len(new_file_idxs)))[0], 'g*', markersize=10)
    # plt.plot(end_file_idxs, np.ones((1,len(end_file_idxs)))[0], 'r*', markersize=10)
    # plt.title('Number of TIF files should be: %i' % len(new_file_markers))


In [ ]:

    onset_evs = [ard_evs[on] for on in onsets]
    offset_evs = [ard_evs[off] for off in offsets]


In [ ]:
    ard_acquisition_evs = []
    ard_frame_onset_times = []
    ard_frame_offset_times = []
    ard_onset_idxs = []
    ard_offset_idxs = []

    ard_file_durs = []
    ard_file_trigger_times = []
    ard_file_trigger_vals = []
    for sidx,eidx in zip(new_file_idxs, end_file_idxs):

        curr_acquisition_evs = ard_evs[sidx:eidx+1]
        ftimes = [e[1] for e in curr_acquisition_evs]

        curr_onset_evs = [ev for ev in onset_evs if ev[1]>=curr_acquisition_evs[0][1] and ev[1]<=curr_acquisition_evs[-1][1]]
        curr_offset_evs = [ev for ev in offset_evs if ev[1]>=curr_acquisition_evs[0][1] and ev[1]<=curr_acquisition_evs[-1][1]]

        curr_onset_times = [e[1] for e in curr_onset_evs]
        curr_offset_times = [e[1] for e in curr_offset_evs]

        curr_onset_idxs = [i for (i,ev) in enumerate(onset_evs) if ev[1]>=curr_acquisition_evs[0][1] and ev[1]<=curr_acquisition_evs[-1][1]]
        curr_offset_idxs = [i for (i,ev) in enumerate(offset_evs) if ev[1]>=curr_acquisition_evs[0][1] and ev[1]<=curr_acquisition_evs[-1][1]]


        frame_trigger_trange = (max(np.diff(curr_onset_times)) - min(np.diff(curr_onset_times))) / 1E3
        print "%i: Range of frame trigger time stamps: %0.2f - %0.2f ms." % (sidx, min(np.diff(curr_onset_times))/1E3, max(np.diff(curr_onset_times))/1E3)

        frame_durs = np.array(curr_offset_times) - np.array(curr_onset_times)

        ard_acquisition_evs.append(curr_acquisition_evs)
        ard_frame_onset_times.append(curr_onset_times)
        ard_frame_offset_times.append(curr_offset_times)
        ard_onset_idxs.append(curr_onset_idxs)
        ard_offset_idxs.append(curr_offset_idxs)


        ard_file_dur = (ard_times[eidx] - ard_times[sidx])/1E6
        ard_file_durs.append(ard_file_dur)

        ard_frame_tval = ard_codes[sidx:eidx+1]
        ard_frame_ttime = ard_times[sidx:eidx+1]
        ard_file_trigger_vals.append(ard_frame_tval)
        ard_file_trigger_times.append(ard_frame_ttime)
